# Evaluation for the 2 Object Spring Experiments
This notebook can be used to load and analyze the results generated by running `training_2ObjectSpring.py`. The notebook is meant to evaluate multiple experiments together and compute the average Performances over all experiments. 

The path to the the folder containing the experiments needs to be specified under `path_experiment`. The folder set here needs to contain the subfolders with the experiments, where those subfolders need to contain `ckpt.pth` and `./hydra/`.

In [ ]:
%cd ..

In [ ]:
import os
import yaml
import torch
from torchvision import utils
from models.sceneRepresentation import Scene
from dataset.dataset import ImageDataset_paig
from util.util import compute_psnr
import matplotlib.pyplot as plt

if torch.cuda.is_available():
    device = "cuda"
else:
    device = "cpu"

## Quantitative evaluation

In [ ]:
# Set the path to the experiments to analyze.
# The folder specified here needs to contain the subfolders which contain `ckpt.pth` and `./hydra/` 
path_experiments = os.path.join(
    os.path.abspath(''),
    'experiments',
    '2023-01-24',
    'Spring'
)

In [ ]:
param_errors = []
psnrs = []

# Get all the experiments
for path_experiment in os.scandir(path_experiments):
    # Load Config
    path_conf = os.path.join(path_experiment, '.hydra','config.yaml')
    with open(path_conf) as f:
        cfg = yaml.safe_load(f)

    print("Doing batch_idx: ", cfg['data']['batch_idx'])

    # Load Model
    model = Scene(**cfg['scene']['background'])

    model.add_2ObjectsSpring(
        **cfg['scene']['local_representation']
    )

    path_ckpt = os.path.join(path_experiment, 'ckpt.pth')
    model.load_state_dict(torch.load(path_ckpt))

    model.to(device)

    # Load Data
    data_path = os.path.join(os.path.abspath(''), 'data',cfg['data']['path_data'])

    data = ImageDataset_paig(
            path_data=data_path,
            batch_idx=cfg['data']['batch_idx'],
            use_subsampling=False,
        )
    H, W = data.get_image_dim()

    # Compute Parameter Errors
    true_k = 2.0
    true_l = 12.0
    rel_error_k = torch.abs(true_k - model.local_representation.ode.k.data) / true_k
    rel_error_l = torch.abs(true_l - W*model.local_representation.ode.eq_distance.data) / true_l

    # Compute PSNR and IoU
    tspan = data.t_steps.to(device)
    model.update_trafo(tspan)
    output = model.render_image(W, H)
    psnr = compute_psnr(output['Image'].cpu(), data.get_full_images())

    param_errors.append(rel_error_k)
    param_errors.append(rel_error_l)
    psnrs.append(psnr)

    mean_errors = torch.mean(torch.tensor([rel_error_k, rel_error_l]))
    print(f"Mean Errors: {mean_errors}, PSNR: {psnr}")

    print("Done")
    print("====================================================")

avg_param_error = torch.mean(torch.tensor(param_errors))
median_param_error = torch.median(torch.tensor(param_errors))
avg_psnr = torch.mean(torch.tensor(psnrs))
print("Results:")
print(f"Avg Param Error: {avg_param_error}, Median Param Error: {median_param_error} Avg PSNR: {avg_psnr}")


## Render Images
The following code creates images for a specific experiment. The folder containing `ckpt.pth` and `./hydra/` for this experiment needs to be specified under `path_experiment`.

In [ ]:
path_experiment = os.path.join(
    os.path.abspath(''),
    'experiments',
    '2023-01-24',
    'Spring',
    '09-45-53_seq0'
)

path_conf = os.path.join(path_experiment, '.hydra','config.yaml')
with open(path_conf) as f:
    cfg = yaml.safe_load(f)

if torch.cuda.is_available():
    device = "cuda"
else:
    device = "cpu"

model = Scene(**cfg['scene']['background'])

model.add_2ObjectsSpring(
    **cfg['scene']['local_representation']
)

path_ckpt = os.path.join(path_experiment, 'ckpt.pth')
model.load_state_dict(torch.load(path_ckpt))

model.to(device)
print("Moved to device")

data_path = os.path.join(os.path.abspath(''), 'data',cfg['data']['path_data'])

render_dataset = ImageDataset_paig(
        path_data=data_path,
        batch_idx=cfg['data']['batch_idx'],
    )

tspan_render = render_dataset.t_steps.to(device)

model.eval()
model.update_trafo(tspan_render)
H, W = render_dataset.get_image_dim()
output = model.render_image(W, H)
ims = output["Image"].cpu()

plt.imshow(ims[0].cpu().numpy())
plt.show()
plt.imshow(ims[25].cpu().numpy())

path_folder_rendering = os.path.join(path_experiment, 'renderings')
os.makedirs(path_folder_rendering)

# Save individual images
for i in range(len(tspan_render)):
    path = os.path.join(path_folder_rendering, f"{i:02}_ours.jpg")
    cur_im = ims[i].permute(2, 0, 1)
    utils.save_image(cur_im, path)

    path = os.path.join(path_folder_rendering, f"{i:02}_gt.jpg")
    cur_im_gt = render_dataset.get_full_images(i).permute(2, 0, 1)


print("PSNR: ", compute_psnr(ims, render_dataset.get_full_images()))
print("Error k: ", torch.abs(2.0 - model.local_representation.ode.k.data) / 2.0)
print("Error l: ", torch.abs(12.0 - W*model.local_representation.ode.eq_distance.data) / 12.0)

## Render segmentation masks
Creates a plot of the segmentation masks used for the segmentation loss on the first frame

In [ ]:
ims = render_dataset.get_full_images(0)
masks = render_dataset.get_full_mask(0)

blend = torch.tensor([0., 0.5, 0.]).unsqueeze(0).unsqueeze(0).repeat(ims.shape[0], ims.shape[1], 1)
mask = masks['masks1'].float().unsqueeze(-1) * 0.6
blended = mask*blend + (1-mask)*ims
plt.imshow(blended)
plt.show()

path = os.path.join(path_folder_rendering, f"init1.jpg")
utils.save_image(blended.permute(2, 0, 1), path)

blend = torch.tensor([0.5, 0., 0.]).unsqueeze(0).unsqueeze(0).repeat(ims.shape[0], ims.shape[1], 1)
mask = masks['masks2'].float().unsqueeze(-1) * 0.6
blended = mask*blend + (1-mask)*ims
plt.imshow(blended)
path = os.path.join(path_folder_rendering, f"init2.jpg")
utils.save_image(blended.permute(2, 0, 1), path)